In [33]:
pip install numpy pandas tensorflow torch

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [34]:
pip install torch torchvision torchaudio


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [35]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [36]:
!gdown --id 16MIleqoIr1vYxlGk4GKnGmrsCPuWkkpT

/Users/humayrajeba/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/humayrajeba/Library/Python/3.9/lib/python/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=16MIleqoIr1vYxlGk4GKnGmrsCPuWkkpT
To: /Users/humayrajeba/Documents/Research work/ECG-LSTM/ECG5000.zip
100%|██████████████████████████████████████| 10.6M/10.6M [00:02<00:00, 4.43MB/s]


In [37]:
!unzip -qq ECG5000.zip

replace ECG5000.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [57]:
import sys
import numpy as np
import pandas as pd
import torch

# Display Python version
print(f"Python version: {sys.version}")

# Display package versions
print(f"numpy version: {np.__version__}")
print(f"pandas version: {pd.__version__}")
print(f"torch version: {torch.__version__}")


Python version: 3.9.6 (default, Feb  3 2024, 15:58:27) 
[Clang 15.0.0 (clang-1500.3.9.4)]
numpy version: 1.26.4
pandas version: 2.2.2
torch version: 2.4.0


In [83]:
import torch
import copy
import numpy as np
from scipy.io import arff
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split

from torch import nn, optim
import torch.nn.functional as F


%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
rng = torch.manual_seed(RANDOM_SEED)

In [84]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [86]:

from scipy.io import arff
import pandas as pd

# Load the training ARFF file
with open('ECG5000_TRAIN.arff') as f:
    data, meta = arff.loadarff(f)
    train = pd.DataFrame(data)

# Load the test ARFF file
with open('ECG5000_TEST.arff') as f:
    data, meta = arff.loadarff(f)
    test = pd.DataFrame(data)

# Display the first few rows of the training data
print(train.head())

       att1      att2      att3      att4      att5      att6      att7  \
0 -0.112522 -2.827204 -3.773897 -4.349751 -4.376041 -3.474986 -2.181408   
1 -1.100878 -3.996840 -4.285843 -4.506579 -4.022377 -3.234368 -1.566126   
2 -0.567088 -2.593450 -3.874230 -4.584095 -4.187449 -3.151462 -1.742940   
3  0.490473 -1.914407 -3.616364 -4.318823 -4.268016 -3.881110 -2.993280   
4  0.800232 -0.874252 -2.384761 -3.973292 -4.338224 -3.802422 -2.534510   

       att8      att9     att10  ...    att132    att133    att134    att135  \
0 -1.818286 -1.250522 -0.477492  ...  0.792168  0.933541  0.796958  0.578621   
1 -0.992258 -0.754680  0.042321  ...  0.538356  0.656881  0.787490  0.724046   
2 -1.490659 -1.183580 -0.394229  ...  0.886073  0.531452  0.311377 -0.021919   
3 -1.671131 -1.333884 -0.965629  ...  0.350816  0.499111  0.600345  0.842069   
4 -1.783423 -1.594450 -0.753199  ...  1.148884  0.958434  1.059025  1.371682   

     att136    att137    att138    att139    att140  target  
0  0.2